In [ ]:
# Description: This program predicts the price of Bitcoin for the next 30 days

In [2]:
#Import the libraries
!pip install -U pandas
!pip install -U pandas_datareader

import pandas as pd
import numpy as np
import datetime as dt
import pandas_datareader as web

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#Load the data
crypto_currency = 'BTC'
against_currency = 'USD'

start = dt.datetime(2016, 1, 1)
end = dt.datetime.now()

data = web.DataReader(
    f'{crypto_currency}-{against_currency}',
    'yahoo',
    start,
    end
)

In [4]:
#Show the first 7 lines of data
data.head(7)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2016-01-01,436.246002,427.515015,430.721008,434.334015,36278900,434.334015
2016-01-02,436.062012,431.869995,434.622009,433.437988,30096600,433.437988
2016-01-03,433.743011,424.705994,433.578003,430.010986,39633800,430.010986
2016-01-04,434.516998,429.084015,430.061005,433.091003,38477500,433.091003
2016-01-05,434.182007,429.675995,433.069000,431.959991,34522600,431.959991
2016-01-06,431.855988,426.341003,431.855988,429.105011,34042500,429.105011
2016-01-07,458.765991,429.076996,430.010986,458.048004,87562200,458.048004


In [15]:
# Remove extra columns
df = pd.DataFrame()
df['Price'] = data['Close'].values

In [14]:
#Show the first 7 lines of data
df.head(7)

,Price
0,434.334015
1,433.437988
2,430.010986
3,433.091003
4,431.959991
5,429.105011
6,458.048004


In [16]:
# A variable for predicting 'n' days out into future
prediction_days = 30 # n = prediction_days = 30

#Create another column shifted 'n' units up
df['Prediction'] = df[['Price']].shift(-prediction_days)

In [17]:
#Show the first 7 rows of the new data
df.head(7)

,Price,Prediction
0,434.334015,368.766998
1,433.437988,373.056000
2,430.010986,374.447998
3,433.091003,369.949005
4,431.959991,389.593994
5,429.105011,386.549011
6,458.048004,376.522003


In [18]:
#Show the last 7 rows
df.tail(7)

,Price,Prediction
2355,22487.388672,NaN
2356,22206.792969,NaN
2357,22572.839844,NaN
2358,20381.650391,NaN
2359,20471.482422,NaN
2360,19017.642578,NaN
2361,19466.316406,NaN


In [48]:
#Create the indipendent dataset
X = np.array(df.drop(['Prediction'], 1))
X = X[:len(df)-prediction_days]
print(X)

[[  434.33401489]
 [  433.43798828]
 [  430.01098633]
 ...
 [28720.27148438]
 [30314.33398438]
 [29200.74023438]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [49]:
#Create the dependent dataset
Y = np.array(df['Prediction'])
Y = Y[:-prediction_days]
print(Y)

[  368.76699829   373.05599976   374.44799805 ... 20471.48242188
 19017.64257812 19466.31640625]


In [52]:
#Split the data into 80% training and 20% testing
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=False)

In [53]:
# Set the prediction_days_array equal to the last 30 rows from the original dataset
prediction_days_array = np.array(df.drop(['Prediction'], 1))[-prediction_days:]
print(prediction_days_array)

[[29432.2265625 ]
 [30323.72265625]
 [29098.91015625]
 [29655.5859375 ]
 [29562.36132812]
 [29267.22460938]
 [28627.57421875]
 [28814.90039062]
 [29445.95703125]
 [31726.390625  ]
 [31792.31054688]
 [29799.08007812]
 [30467.48828125]
 [29704.390625  ]
 [29832.9140625 ]
 [29906.66210938]
 [31370.671875  ]
 [31155.47851562]
 [30214.35546875]
 [30111.99804688]
 [29083.8046875 ]
 [28360.81054688]
 [26762.6484375 ]
 [22487.38867188]
 [22206.79296875]
 [22572.83984375]
 [20381.65039062]
 [20471.48242188]
 [19017.64257812]
 [19466.31640625]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [54]:
from sklearn.svm import SVR

#Create and train the Support Vector Machine (Regressio) using radial basis function
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.00001)
svr_rbf.fit(x_train, y_train)

SVR(C=1000.0, gamma=1e-05)

In [55]:
#Test the model
svr_rbf_confidence = svr_rbf.score(x_test, y_test)
print("svr_rbf accuracy: ", svr_rbf_confidence)

svr_rbf accuracy:  -11.004062629197294


In [56]:
#Print the predicted values
svm_prediction = svr_rbf.predict(x_test)
print(svm_prediction)
print()
print(y_test)

[10931.82697144 10931.82697144 10931.82697144 10931.82697144
 10931.82697144 10931.82697144 10931.82697144 10931.82697144
 10931.82697144 10931.82697144 10931.82697144 10931.82697144
 10931.82697144 10931.82697144 10931.82697144 10931.82697144
 10931.82697144 10931.82697144 10931.82697144 10931.82697144
 10931.82697144 10931.82697144 10931.82697144 10931.82697144
 10931.82697144 10931.82697144 10931.82697144 10931.82697144
 10931.82697144 10931.82697144 10931.82697144 10931.82697144
 10931.82697144 10931.82697144 10931.82697144 10931.82697144
 10931.82697144 10931.82697144 10931.82697144 10931.82697144
 10931.82697144 10931.82697144 10931.82697144 10931.82697144
 10931.82697144 10931.82697144 10931.82697144 10931.82697144
 10931.82697144 10931.82697144 10931.82697144 10931.82697144
 10931.82697144 10931.82697144 10931.82697144 10931.82697144
 10931.82697144 10931.82697144 10931.82697144 10931.82697144
 10931.82697144 10931.82697144 10931.82697144 10931.82697144
 10931.82697144 10931.82

In [57]:
# Print the model predictions for the next 'n=30' days
svm_prediction = svr_rbf.predict(prediction_days_array)
print(svm_prediction)
print()
print(df.tail(prediction_days))

[12085.73628151 11900.74023565 12824.54980661 11402.35197776
 11682.69616266 12480.43404669 11816.5097754  12674.26437491
 12046.33387562 12051.43627202 12574.38946515 11116.14223418
 12196.86126654 11280.58656829 11082.26755492 11053.95199746
 11021.50229557 11275.84325555 11577.71928704 11300.18331311
 12851.53413389 11048.07690498 11750.11846903 11683.21460735
 10988.77311596 12156.03032573 10935.18096867 10933.4555166
 17124.78489325 15535.99675247]

             Price  Prediction
2332  29432.226562         NaN
2333  30323.722656         NaN
2334  29098.910156         NaN
2335  29655.585938         NaN
2336  29562.361328         NaN
2337  29267.224609         NaN
2338  28627.574219         NaN
2339  28814.900391         NaN
2340  29445.957031         NaN
2341  31726.390625         NaN
2342  31792.310547         NaN
2343  29799.080078         NaN
2344  30467.488281         NaN
2345  29704.390625         NaN
2346  29832.914062         NaN
2347  29906.662109         NaN
2348  31370.67